In [1]:
!pip install -U pytorch-pretrained-bert
!pip install transformers

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 778kB 2.9MB/s 
     |████████████████████████████████| 3.0MB 14.6MB/s 
     |████████████████████████████████| 1.1MB 35.9MB/s 
     |████████████████████████████████| 890kB 43.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=cd2baeb8995e672b753c0b65c78a67c5ea43995519859054b5501661e7af74f4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [20]:
import torch
#from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertConfig, AutoModelWithLMHead
from scipy.spatial.distance import cosine
import numpy as np
import copy
from transformers import AutoTokenizer, AutoModel, AutoConfig
from torch import nn

### Load Model

In [85]:
# bert-base-uncased
bertconfig = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
basetokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
basemodel = BertForMaskedLM.from_pretrained('bert-base-uncased', config = bertconfig)
# Clinical BERT
# https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT
cliconfig = AutoConfig.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", output_hidden_states=True)
clitokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
climodel = BertForMaskedLM.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", config = bioconfig)
# bert-large-uncased
largeconfig = BertConfig.from_pretrained('bert-large-uncased', output_hidden_states=True)
largetokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
largemodel = BertForMaskedLM.from_pretrained('bert-large-uncased', config = largeconfig)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initi

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert-base-uncased

In [5]:
tokenizer = basetokenizer
model = basemodel

## Step by step prediction

In [6]:
# [MASK]: respiratory
text = '[CLS] Sputum is usually yellow or clear and different from the green sputum that is more often associated with lower [MASK] tract infections. [SEP]'
TOPK = 10

In [7]:
tokenized_text = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [0] * len(tokenized_text)
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
# Predict hidden states features for each layer
# https://huggingface.co/transformers/model_doc/bert.html#bertformaskedlm
with torch.no_grad(): # prediction
    outputs = model(tokens_tensor, segments_tensors)

In [8]:
predictions = outputs[0]
hidden_states = outputs[1]

Candidates and Probability of [MASK]

In [9]:
masked_index = tokenized_text.index('[MASK]')
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

print("Candidate - Probability\n")
prob = F.softmax(predictions[0, masked_index], dim = 0)
prob_predicted_values, prob_predicted_indices = prob.topk(TOPK, dim=0, largest=True, sorted=True)
word_cands = []
for i in range(TOPK):
  cur_predicted_token = tokenizer.convert_ids_to_tokens([prob_predicted_indices[i].item()])[0]
  tmp = []
  tmp.append(cur_predicted_token)
  tmp.append(prob_predicted_values[i].item())
  word_cands.append(tmp)
  print(tmp[0], tmp[1])

Candidate - Probability

respiratory 0.9626149535179138
gi 0.021592240780591965
reproductive 0.006219645030796528
metabolic 0.0009671205189079046
##mental 0.0006188023835420609
lying 0.0004803779302164912
immune 0.00044372095726430416
neural 0.0002540722780395299
optic 0.00025145182735286653
nervous 0.00019847533258143812


In [10]:
word_cands

[['respiratory', 0.9626149535179138],
 ['gi', 0.021592240780591965],
 ['reproductive', 0.006219645030796528],
 ['metabolic', 0.0009671205189079046],
 ['##mental', 0.0006188023835420609],
 ['lying', 0.0004803779302164912],
 ['immune', 0.00044372095726430416],
 ['neural', 0.0002540722780395299],
 ['optic', 0.00025145182735286653],
 ['nervous', 0.00019847533258143812]]


Create the word vectors by summing together the last four layers from hidden states

https://colab.research.google.com/drive/1yFphU6PW9Uo6lmDly_ud9a6c4RCYlwdX#scrollTo=eYXUwiG0yhBS

In [11]:
# Concatenate the tensors for all layers. 
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings.size()

torch.Size([13, 1, 29, 768])

In [12]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings.size()

torch.Size([13, 29, 768])

In [13]:
token_embeddings = token_embeddings.permute(1,0,2)

In [14]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.
# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 29 x 768


In [15]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 sp
2 ##ut
3 ##um
4 is
5 usually
6 yellow
7 or
8 clear
9 and
10 different
11 from
12 the
13 green
14 sp
15 ##ut
16 ##um
17 that
18 is
19 more
20 often
21 associated
22 with
23 lower
24 [MASK]
25 tract
26 infections
27 .
28 [SEP]


In [16]:
token_vecs_sum[0][:5]

tensor([-2.0478, -0.3244, -0.6018, -2.4874, -0.1112])

In [18]:
len(token_vecs_sum)

29

## Detector for single sentence

In [76]:
class SentDetector():
  def __init__(self, model, tokenizer, k = 5):
    self.model = model
    self.tokenizer = tokenizer
    self.topK = k

  def generate_cands(self, tokenized_text):
    '''
    Input: a tokenized sentence with [MASK] and specific model
    Output: hidden_states - Hidden states of the model at the output of each layer plus the initial embedding outputs.
            word_cands -  a list of candidate words and their probability
    '''
    # tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [0] * len(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    # Predict hidden states features for each layer
    # https://huggingface.co/transformers/model_doc/bert.html#bertformaskedlm
    with torch.no_grad(): # prediction
        outputs = self.model(tokens_tensor, segments_tensors)

    predictions = outputs[0]
    hidden_states = outputs[1]
    masked_index = tokenized_text.index('[MASK]')
    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = self.tokenizer.convert_ids_to_tokens([predicted_index])[0]

    #print("Candidate - Probability\n")
    prob = F.softmax(predictions[0, masked_index], dim = 0)
    prob_predicted_values, prob_predicted_indices = prob.topk(self.topK, dim=0, largest=True, sorted=True)
    word_cands = []
    for i in range(self.topK):
      cur_predicted_token = self.tokenizer.convert_ids_to_tokens([prob_predicted_indices[i].item()])[0]
      tmp = [cur_predicted_token, prob_predicted_values[i].item()]
      word_cands.append(tmp)
      #print(tmp[0], tmp[1])
    return hidden_states, word_cands

  def cand_embeddings(self, word_cands):
    '''
    Get output embeddings of prediction words from the model
    Output: a list of token and their embeddings
    '''
    output_embeddings = self.model.get_output_embeddings()
    embedding_lst = []
    for i in range(len(word_cands)):
      cur_emb = output_embeddings.weight[self.tokenizer.convert_tokens_to_ids(word_cands[i][0])]
      tmp = [word_cands[i][0], cur_emb]
      embedding_lst.append(tmp)
    return embedding_lst

  def generate_raw_hidden(self):
    '''
    Generate embeddings for each token in the raw sentence
    Output: hidden_states - Hidden-states of the model at the output of each layer plus the initial embedding outputs.
    '''
    #tokenized_text = self.tokenizer.tokenize(self.raw_text)
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(self.tokenized_raw_text)
    segments_ids = [0] * self.length
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    # Predict hidden states features for each layer
    with torch.no_grad(): # prediction
        outputs = self.model(tokens_tensor, segments_tensors)
    return outputs[1]

  def raw_embeddings(self):
    '''
    Output: list of tokens and their "embeddings" from original hidden states
    Word embedding: sum over the last four layers (from BERT tutorial)
    '''
    hidden_states = self.generate_raw_hidden()
    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim = 0), dim = 1)
    # switch around the "layers" and "tokens" dimensions with permute.
    token_embeddings = token_embeddings.permute(1,0,2)
    token_vecs_sum = []
    # `token_embeddings` is a [#tokens x 12 x 768] tensor.
    # For each token in the sentence...
    for token in token_embeddings:
        # `token` is a [12 x 768] tensor
        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)
        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)
    return token_vecs_sum

  def mask_ith_token(self, i):
    '''
    Output: text with the ith token masked
    '''
    # tokenized_text = tokenizer.tokenize(text)
    res = copy.deepcopy(self.tokenized_raw_text)
    res[i] = '[MASK]' 
    return res

  def cal_similarity(self, candidates, raw_emb):
    '''
    Input: masked sentence
    Output: a list of cosine similarities between each candidate and the original word
    '''
    similarities = []
    embedding_lst = self.cand_embeddings(candidates)
    for word, cur_emb in embedding_lst:
      cos_sim = 1 - cosine(raw_emb.numpy(), cur_emb.detach().numpy())
      similarities.append([word,cos_sim])

    return similarities

  def changeModel(self, model, tokenizer):
    self.model = model
    self.tokenizer = tokenizer

  def changeK(self, k):
    self.topK = k

  def run(self, text, verbose = False):
    self.raw_text = text
    self.tokenized_raw_text = self.tokenizer.tokenize(self.raw_text)
    self.length = len(self.tokenized_raw_text)

    #similarity scores
    simi_scores = [1]
    raw_emb = self.raw_embeddings()

    for i in range(1, self.length - 1):
      cur_masked_tokens = self.mask_ith_token(i)
      hidden, candidates = self.generate_cands(cur_masked_tokens)
      if verbose:
        print(cur_masked_tokens)
      similarities = self.cal_similarity(candidates, raw_emb[i])
      # prob * similarity
      simi_scores.append(np.sum([(candidates[i][1] * similarities[i][1]) for i in range(self.topK)]))
      if verbose:
        print("Candidates - Probability")
        print(candidates)
        print("Similarity")
        print(similarities)
        print('\n')
    #min_vals = torch.from_numpy(simi_scores).topk(3, largest = False)
    min_vals =  np.argsort(simi_scores)[:self.topK]
    suspect_idx = simi_scores.index(np.min(simi_scores))
    #print(simi_scores)
    print("The suspect words are: " )
    #print("\nThe suspect word is : " , tokenized_text[suspect_idx])
    i = 1
    for idx in min_vals:
      print(i, self.tokenized_raw_text[idx], simi_scores[idx])
      i += 1
      #print(tokenized_text[simi_scores[val]]+", ")
    return


test

In [77]:
text = '[CLS] It has many casual and can affect any organ system. [SEP]' 
test = SentDetector(basemodel, basetokenizer, TOPK)
test.run(text)

The suspect words are: 
1 casual 0.0022504691182495488
2 organ 0.03531347074744821
3 affect 0.054766716123304096
4 many 0.06492909023989726
5 system 0.0930543173501405
6 can 0.10531885531632658
7 has 0.11887342322209289
8 it 0.13812981409874328
9 . 0.18646535354910282
10 any 0.24983780810365538


In [75]:
# change to Clinical BERT model
test.changeModel(climodel, clitokenizer)
test.run(text)

The suspect words are: 
1 casual 0.01835374096823243
2 affect 0.026007123283669803
3 it 0.05588177659719311
4 many 0.0708836168603651
5 can 0.10115014722043478
6 system 0.11092818411757911
7 and 0.12286282719746551
8 organ 0.1274559941677678
9 any 0.1288416774451855
10 has 0.15195791119996596


## ipywidgets


In [61]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interactive, Layout, HTML, Label
from IPython.display import display
from scipy.interpolate import interp1d

### widgets

In [62]:
text = widgets.Text()
display(text)

def print_value(sender):
  print(sender.value)

text.on_submit(print_value)

Text(value='')

In [63]:
text_area = widgets.Textarea(
    value='Hello World',
    placeholder='Type something',
    description='Your text:',
    disabled=False
)

In [64]:
btn = widgets.Button(description = "Submit", tooltip = 'Click to submit')
def btn_click(sender):
  print('Text successfully submitted!')
  print_value(text)
btn.on_click(btn_click)
box = widgets.Box([text,btn])
display(box)

Box(children=(Text(value=''), Button(description='Submit', style=ButtonStyle(), tooltip='Click to submit')))

In [65]:
caption = widgets.Label(value='Please select TOPK.')
drpbx = widgets.Dropdown(options=[("1", 1), ("2", 2), ("5", 5)], index=0, value=2, label="2")

def chosen(_):
    print("Selected index:{}, value:{}, label:{}".format(drpbx.index, drpbx.value, drpbx.label))

drpbx.observe(chosen, names="value")
display(caption, drpbx)

Label(value='Please select TOPK.')

Dropdown(index=1, options=(('1', 1), ('2', 2), ('5', 5)), value=2)

Selected index:2, value:5, label:5


In [80]:
caption = widgets.Label(value='Sliders')
sliders1, slider2 = widgets.IntSlider(description='Slider 1'),\
                	widgets.IntSlider(description='Slider 2')
l = widgets.link((sliders1, 'value'), (slider2, 'value'))
display(caption, sliders1, slider2)

Label(value='Sliders')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [81]:
from ipywidgets import Button, HBox
display(
    HBox([
        Button(description='Primary Button', button_style='primary'),
        Button(description='Success Button', button_style='success'),
        Button(description='Info Button', button_style='info'),
        Button(description='Warning Button', button_style='warning'),
        Button(description='Danger Button', button_style='danger')
    ]))

### Single-model Detector


In [79]:
text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='Your text:',
    disabled=False
    )

btn = widgets.Button(
    description = "OK", 
    tooltip = 'Click to submit'
    )

drpd = widgets.Dropdown(
    options=[("1", 1), ("2", 2), ("3", 3), ("5", 5), ("10", 10)], 
    index=0, 
    value=1,
    description='Top K:',
    disabled=False,
    )

caption = widgets.Label(
    value = 'Word appropriateness detector for single sentence.'
    )

detector = SentDetector(basemodel, basetokenizer)

def get_k(sender):
  TOPK = sender.observe(chosen, names="value")

def callback(sender):
  print('Text successfully submitted!')
  TOPK = chosen(drpd)
  raw_text = '[CLS] ' + sender.value + ' [SEP]'
  # print(ori_text)
  # print(TOPK)
  detector.changeK(TOPK)
  detector.run(raw_text)

def btn_click(sender):
  callback(text)

def chosen(_):
  return drpd.value

drpd.observe(chosen, names="value")
btn.button_style='primary'

#text.on_submit(callback)
btn.on_click(btn_click)

display(caption, text, drpd, btn)

Label(value='Word appropriateness detector for single sentence.')

Text(value='Hello World', description='Your text:', placeholder='Type something')

Dropdown(description='Top K:', options=(('1', 1), ('2', 2), ('3', 3), ('5', 5), ('10', 10)), value=1)

Button(button_style='primary', description='OK', style=ButtonStyle(), tooltip='Click to submit')

Text successfully submitted!
The suspect words are: 
1 cute 0.010036780340255746
2 bilateral 0.012263965784352002
3 larger 0.01941094052036111
4 airspace 0.04311686577852297
5 diagnosis 0.06429604935333656
6 subset 0.0811937392028782
7 . 0.10261349746239871
8 a 0.11668218369054956
9 differential 0.12463457253199668
10 ##ification 0.12890247093414056


Acute cholangitis, or ascending cholangitis, is a form of cholangitis and refers to the acute bacterial infection of the biliary three. # bacterial

It has many causes and can affect any organ system.

A cute bilateral airspace opacification is a subset of the larger differential diagnosis for airspace opacification.

### Multi-model Detector

In [88]:
text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='Your text:',
    disabled=False
    )

btn1 = widgets.Button(
    description = "BERT-base", 
    tooltip = 'Click to submit',
    button_style = 'info'
    )

btn2 = widgets.Button(
    description = "Clinical BERT", 
    tooltip = 'Click to submit',
    button_style = 'success'
    )

btn3 = widgets.Button(
    description = "BERT-large", 
    tooltip = 'Click to submit',
    button_style = 'primary'
    )

drpd = widgets.Dropdown(
    options=[("1", 1), ("2", 2), ("3", 3), ("5", 5), ("10", 10)], 
    index=0, 
    value=1,
    description='Top K:',
    disabled=False,
    )

caption = widgets.Label(
    value = 'Word appropriateness detector for single sentence.'
    )

detector = SentDetector(basemodel, basetokenizer)

def get_k(sender):
  TOPK = sender.observe(chosen, names="value")

def callback(sender):
  TOPK = chosen(drpd)
  raw_text = '[CLS] ' + sender.value + ' [SEP]'
  # print(ori_text)
  # print(TOPK)
  detector.changeK(TOPK)
  detector.run(raw_text)

def btn1_click(sender):
  detector.changeModel(basemodel, basetokenizer)
  print('===========================\nText successfully submitted to BERT-base!')
  callback(text)

def btn2_click(sender):
  detector.changeModel(climodel, clitokenizer)
  print('===========================\nText successfully submitted to Clinical BERT!')
  callback(text)

def btn3_click(sender):
  detector.changeModel(largemodel, largetokenizer)
  print('===========================\nText successfully submitted to BERT-large!')
  callback(text)


def chosen(_):
  return drpd.value

drpd.observe(chosen, names="value")

#text.on_submit(callback)
btn1.on_click(btn1_click)
btn2.on_click(btn2_click)
btn3.on_click(btn3_click)

display(caption, text, drpd, btn1, btn3, btn2)


Label(value='Word appropriateness detector for single sentence.')

Text(value='Hello World', description='Your text:', placeholder='Type something')

Dropdown(description='Top K:', options=(('1', 1), ('2', 2), ('3', 3), ('5', 5), ('10', 10)), value=1)

Button(button_style='info', description='BERT-base', style=ButtonStyle(), tooltip='Click to submit')

Button(button_style='primary', description='BERT-large', style=ButtonStyle(), tooltip='Click to submit')

Button(button_style='success', description='Clinical BERT', style=ButtonStyle(), tooltip='Click to submit')

Acute cholangitis, or ascending cholangitis, is a form of cholangitis and refers to the acute bacterial infection of the biliary three. # bacterial

It has many casual and can affect any organ system.

Acute bilateral airspace opacification is a subset of the larger differential diagnosis for airspace opacification.

Abdominal paracentesis, more commonly referred to as an ascitic tap, is a procedure that can be performed to collect peritoneal fluid for analysis or as a therapeutic intervention.